# ADA / Applied Data Analysis
<h2 style="color:#a8a8a8">Homework 4 - Applied Machine Learning<br>
Aimée Montero, Alfonso Peterssen, Cyriaque Brousse</h2>

## Assignment description
In this homework we will gain experience on Applied Machine Learning, exploring an interesting dataset about soccer players and referees.
You can find all the data in the `CrowdstormingDataJuly1st.csv` file, while you can read a thorough [dataset description here](DATA.md).
Given that the focus of this homework is Machine Learning, I recommend you to first take a look at [this notebook](http://nbviewer.jupyter.org/github/mathewzilla/redcard/blob/master/Crowdstorming_visualisation.ipynb)
containing a solid work in pre-processing + visualization of the given dataset. You are *not* allowed to just copy/paste the pre-processing steps
performed by the notebook authors -- you are still supposed to perform your own data analysis for the homework. Still, I'm confident that consulting first
the work done by expert data analysts will speed up tangibly your effort (i.e., they have already found for you many glitches in the data :)


### Assignment
1. Train a `sklearn.ensemble.RandomForestClassifier` that given a soccer player description outputs his skin color. Show how different parameters 
passed to the Classifier affect the overfitting issue. Perform cross-validation to mitigate the overfitting of your model. Once you assessed your model,
inspect the `feature_importances_` attribute and discuss the obtained results. With different assumptions on the data (e.g., dropping certain features even
before feeding them to the classifier), can you obtain a substantially different `feature_importances_` attribute?

  *BONUS*: plot the learning curves against at least 2 different sets of parameters passed to your Random Forest. To obtain smooth curves, partition
your data in at least 20 folds. Can you find a set of parameters that leads to high bias, and one which does not?

2. Aggregate the referee information grouping by soccer player, and use an unsupervised learning technique to cluster the soccer players in 2 disjoint
clusters. Remove features iteratively, and at each step perform again the clustering and compute the silhouette score -- can you find a configuration of features with high silhouette
score where players with dark and light skin colors belong to different clusters? Discuss the obtained results.

## Part 1 - Familiarizing ourselves with the data

## Data description

From a company for sports statistics, we obtained data and profile photos from all soccer players (N = 2053) playing in the first male divisions of England, Germany, France and Spain in the 2012-2013 season and all referees (N = 3147) that these players played under in their professional career. We created a dataset of player–referee dyads including the number of matches players and referees encountered each other and our dependent variable, the number of red cards given to a player by a particular referee throughout all matches the two encountered each other.
 
Player photos were available from the source for 1586 out of 2053 players. Players’ skin tone was coded by two independent raters blind to the research question who, based on their profile photo, categorized players on a 5-point scale ranging from “very light skin” to “very dark skin” with “neither dark nor light skin” as the center value. 

Additionally, implicit bias scores for each referee country were calculated using a race implicit association test (IAT), with higher values corresponding to faster white | good, black | bad associations. Explicit bias scores for each referee country were calculated using a racial thermometer task, with higher values corresponding to greater feelings of warmth toward whites versus blacks. Both these measures were created by aggregating data from many online users in referee countries taking these tests on [Project Implicit](http://projectimplicit.net).

In all, the dataset has a total of 146028 dyads of players and referees. A detailed description of all variables in the dataset can be seen in the list below.

### Variables:

- *playerShort* - short player ID
- *player* - player name
- *club* - player club
- *leagueCountry* - country of player club (England, Germany, France, and Spain)
- *birthday* - player birthday
- *height* - player height (in cm)
- *weight* - player weight (in kg)
- *position* - detailed player position
- *games* - number of games in the player-referee dyad
- *victories* - victories in the player-referee dyad
- *ties* - ties in the player-referee dyad
- *defeats* - losses in the player-referee dyad
- *goals* - goals scored by a player in the player-referee dyad
- *yellowCards* - number of yellow cards player received from referee
- *yellowReds* - number of yellow-red cards player received from referee
- *redCards* - number of red cards player received from referee
- *photoID* - ID of player photo (if available)
- *rater1* - skin rating of photo by rater 1 (5-point scale ranging from “very light skin” to “very dark skin”)
- *rater2* - skin rating of photo by rater 2 (5-point scale ranging from “very light skin” to “very dark skin”)
- *refNum* - unique referee ID number (referee name removed for anonymizing purposes)
- *refCountry* - unique referee country ID number (country name removed for anonymizing purposes)
- *meanIAT* - mean implicit bias score (using the race IAT) for referee country, higher values correspond to faster white | good, black | bad associations
- *nIAT* - sample size for race IAT in that particular country
- *seIAT* - standard error for mean estimate of race IAT
- *meanExp* - mean explicit bias score (using a racial thermometer task) for referee country, higher values correspond to greater feelings of warmth toward whites versus blacks
- *nExp* - sample size for explicit bias in that particular country
- *seExp* - standard error for mean estimate of explicit bias measure

Let's import the required libraries:

In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import sklearn as sk
import datetime
%matplotlib inline

And import the data from the CSV source:

In [93]:
data = pd.read_csv('CrowdstormingDataJuly1st.csv', parse_dates=['birthday'], infer_datetime_format=True)
data.sample(5)

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
118579,sven-schipplock,Sven Schipplock,1899 Hoffenheim,Germany,1988-11-08,185.0,79.0,Center Forward,1,0,...,0.25,2525,8,DEU,0.336628,7749.0,0.000055,0.335967,7974.0,0.000225
39811,julian-riedel,Julian Riedel,Bayer Leverkusen,Germany,1991-08-10,183.0,73.0,Left Fullback,1,0,...,0.25,740,8,DEU,0.336628,7749.0,0.000055,0.335967,7974.0,0.000225
118857,ali-al-habsi,Ali Al Habsi,Wigan Athletic,England,1981-12-30,194.0,88.0,Goalkeeper,1,1,...,NaN,2530,44,ENGL,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037
43085,richard-wright,Richard Wright,Manchester City,England,1977-11-05,189.0,87.0,NaN,2,1,...,0.25,783,44,ENGL,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037
30278,kamel-ghilas,Kamel Ghilas,Stade Reims,France,1984-03-09,177.0,70.0,Attacking Midfielder,1,0,...,NaN,553,99,BEN,0.255357,238.0,0.002299,0.196581,234.0,0.014588


In [94]:
def discretize(column):
    le = preprocessing.LabelEncoder()
    le.fit(column)
    return le.transform(column)

In [113]:
def feature_engineering(data):
    # First, we follow the intuition that we need to reconcile the `rater1` and `rater2` columns.
    # Indeed, we want a single value on which we will then evaluate the model against.
    # We use the mean as aggregation function:
    data['skin'] = (data.rater1 + data.rater2) / 2
    data = data.drop('rater1', axis=1)
    data = data.drop('rater2', axis=1)
    
    non_features = [
        'player',
    ]

    data = data.drop(non_features, axis=1)
    
    # Let's also drop the `photoID` column, since we don't have access to the photos anyways:
    data = data.drop('photoID', axis=1)
    data = data.drop('Alpha_3', axis=1)
    
    data = data.dropna()
    
    non_numeric_features = [
        'club',
        'leagueCountry',
        'position',
        'playerShort'
    ]
    
    for f in non_numeric_features:
        data[f] = discretize(data[f])
    
    today = datetime.date.today()
    data.birthday = [today.year - d.year for d in data.birthday]
    
    y = data['skin']
    X = data.drop('skin', axis=1)
    
    
    return (X, y)

We notice that the `refCountry` and `Alpha_3` columns are bijective: they contain the same data.

In [114]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
from sklearn import preprocessing

In [115]:
(train_X, train_Y) = feature_engineering(data)
rfc = rfc.fit(train_X, train_Y)

ValueError: Unknown label type: array([[ 0.375],
       [ 0.75 ],
       [ 0.125],
       ..., 
       [ 0.375],
       [ 0.25 ],
       [ 0.125]])

In [ ]:
rfc.predict(train_X)

(115457, 23)


(115457, 23)

In [76]:
data.birthday

0         33
1         34
2         24
3         23
4         25
5         37
6         31
7         30
8         32
9         29
10        32
11        30
12        28
13        34
14        32
15        28
16        29
17        35
18        27
19        28
20        27
21        30
22        28
23        24
24        27
25        28
26        28
27        39
28        35
29        28
          ..
145998    31
145999    30
146000    30
146001    33
146002    29
146003    32
146004    32
146005    34
146006    36
146007    33
146008    30
146009    37
146010    33
146011    23
146012    31
146013    29
146014    33
146015    35
146016    40
146017    35
146018    27
146019    30
146020    40
146021    35
146022    30
146023    36
146024    28
146025    25
146026    26
146027    38
Name: birthday, dtype: int64